# Databricks SQL Fundamentals - Interactive Demo

Welcome! This demo will teach you the fundamentals of Databricks SQL and important concepts you need to know.

---

## 🎯 What is Databricks SQL?

**Databricks SQL** is a SQL interface built on Apache Spark that provides:
* ANSI SQL compliance with Spark extensions
* Direct access to Unity Catalog tables
* Optimized query execution with Photon engine
* Integration with BI tools and dashboards

**Key Difference from Standard SQL:**
Databricks SQL queries are compiled into Spark execution plans, giving you distributed processing power!

---

## 📚 What You'll Learn

1. **SQL Transformations** - SELECT, WHERE, CASE, GROUP BY, ORDER BY
2. **Strict Typing** ⚠️ - Type casting and compatibility gotchas
3. **Store Assignment Policy** ⚠️ - INSERT/CREATE TABLE type enforcement
4. **EXPLAIN Plans** - See how SQL becomes Spark execution
5. **Unity Catalog Integration** - Create and query tables

---

## 📊 Demo Scenario

We'll analyze NYC taxi trip data to create hourly summaries - the same analysis from the PySpark lab, but using pure SQL!

**Let's get started!** 🚀

## 1. Explore Unity Catalog with SQL 🔍

**Unity Catalog** is Databricks' unified governance solution for data and AI assets.

**Three-level namespace:**
* **Catalog** - Top-level container (e.g., `samples`, `main`)
* **Schema** - Database within a catalog (e.g., `nyctaxi`, `default`)
* **Table** - Actual data table (e.g., `trips`)

**Full table reference:** `catalog.schema.table`

Let's explore what's available!

In [0]:
%sql
-- List all catalogs you have access to
SHOW CATALOGS

In [0]:
%sql
-- List all schemas in the samples catalog
SHOW SCHEMAS IN samples

In [0]:
%sql
-- List all tables in the nyctaxi schema
SHOW TABLES IN samples.nyctaxi

In [0]:
%sql
-- See the structure of the trips table
DESCRIBE samples.nyctaxi.trips

In [0]:
%sql
-- Look at sample data
SELECT *
FROM samples.nyctaxi.trips
LIMIT 10

## 2. SQL Transformations 🔄

Let's build our analysis step by step, just like the PySpark lab!

**Transformations we'll apply:**
1. **SELECT** - Choose specific columns (like PySpark's `select()`)
2. **WHERE** - Filter rows (like PySpark's `filter()`)
3. **CASE WHEN** - Add calculated columns (like PySpark's `withColumn()` + `when()`)
4. **GROUP BY** - Aggregate data (like PySpark's `groupBy()`)
5. **ORDER BY** - Sort results (like PySpark's `orderBy()`)

In [0]:
%sql
-- SELECT: Choose only the columns we need
-- This is equivalent to PySpark's df.select()

SELECT 
  tpep_pickup_datetime,
  fare_amount,
  trip_distance
FROM samples.nyctaxi.trips
LIMIT 10

In [0]:
%sql
-- WHERE: Filter for valid trips only
-- This is equivalent to PySpark's df.filter()

SELECT 
  tpep_pickup_datetime,
  fare_amount,
  trip_distance
FROM samples.nyctaxi.trips
WHERE 
  fare_amount > 0
  AND trip_distance > 0
  AND fare_amount < 500
LIMIT 10

In [0]:
%sql
-- Add calculated columns with CASE WHEN
-- This is equivalent to PySpark's withColumn()

SELECT 
  tpep_pickup_datetime,
  fare_amount,
  trip_distance,
  
  -- Extract hour from timestamp (like PySpark's hour() function)
  HOUR(tpep_pickup_datetime) AS pickup_hour,
  
  -- Categorize revenue (like PySpark's when().otherwise())
  CASE 
    WHEN fare_amount < 10 THEN 'Low'
    WHEN fare_amount <= 30 THEN 'Medium'
    ELSE 'High'
  END AS revenue_category
  
FROM samples.nyctaxi.trips
WHERE 
  fare_amount > 0
  AND trip_distance > 0
  AND fare_amount < 500
LIMIT 10

In [0]:
%sql
-- GROUP BY: Aggregate data by hour
-- This is equivalent to PySpark's groupBy().agg()

SELECT 
  HOUR(tpep_pickup_datetime) AS pickup_hour,
  
  -- Aggregation functions
  COUNT(*) AS trip_count,
  ROUND(SUM(fare_amount), 2) AS total_revenue,
  ROUND(AVG(fare_amount), 2) AS avg_fare,
  ROUND(AVG(trip_distance), 2) AS avg_distance
  
FROM samples.nyctaxi.trips
WHERE 
  fare_amount > 0
  AND trip_distance > 0
  AND fare_amount < 500
GROUP BY HOUR(tpep_pickup_datetime)

In [0]:
%sql
-- ORDER BY: Sort results chronologically
-- This is equivalent to PySpark's orderBy()

SELECT 
  HOUR(tpep_pickup_datetime) AS pickup_hour,
  COUNT(*) AS trip_count,
  ROUND(SUM(fare_amount), 2) AS total_revenue,
  ROUND(AVG(fare_amount), 2) AS avg_fare,
  ROUND(AVG(trip_distance), 2) AS avg_distance
FROM samples.nyctaxi.trips
WHERE 
  fare_amount > 0
  AND trip_distance > 0
  AND fare_amount < 500
GROUP BY HOUR(tpep_pickup_datetime)
ORDER BY pickup_hour ASC

### 🎯 Complete Query

Here's the full query with all transformations combined:

```sql
SELECT 
  HOUR(tpep_pickup_datetime) AS pickup_hour,
  COUNT(*) AS trip_count,
  ROUND(SUM(fare_amount), 2) AS total_revenue,
  ROUND(AVG(fare_amount), 2) AS avg_fare,
  ROUND(AVG(trip_distance), 2) AS avg_distance
FROM samples.nyctaxi.trips
WHERE 
  fare_amount > 0
  AND trip_distance > 0
  AND fare_amount < 500
GROUP BY HOUR(tpep_pickup_datetime)
ORDER BY pickup_hour ASC
```

**SQL vs PySpark Comparison:**

| Operation | PySpark | SQL |
|-----------|---------|-----|
| Select columns | `.select()` | `SELECT` |
| Filter rows | `.filter()` | `WHERE` |
| Add columns | `.withColumn()` | `CASE WHEN`, functions |
| Aggregate | `.groupBy().agg()` | `GROUP BY` + aggregates |
| Sort | `.orderBy()` | `ORDER BY` |

## 3. Gotcha #1: Strict Typing ⚠️

**Important:** Databricks SQL uses **strict typing** - operations between incompatible types will fail!

**What is strict typing?**
* Type mismatches cause errors (not automatic conversion)
* You must explicitly cast types
* Prevents silent data corruption

**Common scenarios:**
* Comparing STRING to INT
* Adding STRING to NUMBER
* Joining on mismatched types

Let's see this in action!

In [0]:
%sql
-- This may not fail if ansi_mode is set to false (default in some Databricks environments)
-- In that case, Databricks implicitly casts STRING to INT or DOUBLE for comparison, this is the behaviour in Serverless Workspaces

SELECT 
  '5' > 10 AS comparison_result,
  '5' as orginal_value,
  10 as int_value

-- If ansi_mode = true, this will fail due to type mismatch
-- If ansi_mode = false, this will return false (because '5' is cast to 5)

In [0]:
%sql
-- Solution: Cast the string to INT

SELECT 
  CAST('5' AS INT) > 10 AS comparison_result,
  CAST('5' AS INT) AS casted_value,
  10 AS int_value

-- Now it works! Result: false

In [0]:
%sql
-- Real scenario: passenger_count might be stored as STRING in some systems
-- Let's simulate this

SELECT 
  CAST(passenger_count AS STRING) AS passenger_count_string,
  fare_amount
FROM samples.nyctaxi.trips
WHERE fare_amount > 0
LIMIT 5

In [0]:
%sql
-- This would FAIL if passenger_count was STRING:
-- WHERE passenger_count_string > 2  (comparing STRING to INT)

-- Correct approach: Cast before comparison
SELECT 
  passenger_count,
  fare_amount
FROM samples.nyctaxi.trips
WHERE 
  CAST(passenger_count AS INT) > 2  -- Explicit cast
  AND fare_amount > 0
LIMIT 10

### 🛠️ Common Type Casting Functions

**Casting syntax:**
```sql
CAST(column AS type)
-- or
column::type  -- PostgreSQL-style shorthand
```

**Common types:**
* `INT` / `BIGINT` - Integer numbers
* `DOUBLE` / `DECIMAL(p,s)` - Floating point numbers
* `STRING` - Text
* `BOOLEAN` - True/False
* `DATE` - Date only
* `TIMESTAMP` - Date and time

**Examples:**
```sql
CAST('123' AS INT)                    -- String to integer
CAST(123 AS STRING)                   -- Integer to string
CAST('2024-01-15' AS DATE)           -- String to date
CAST(fare_amount AS DECIMAL(10,2))   -- Specify precision
'123'::INT                            -- Shorthand syntax
```

**⚠️ Best Practice:**
Always cast explicitly when working with different types - don't rely on implicit conversion!

In [0]:
%sql
-- Check data types in your query results
-- Use DESCRIBE to see column types

DESCRIBE QUERY (
  SELECT 
    passenger_count,
    CAST(passenger_count AS STRING) AS passenger_string,
    fare_amount,
    CAST(fare_amount AS INT) AS fare_int
  FROM samples.nyctaxi.trips
  LIMIT 1
)

## 4. Gotcha #2: Store Assignment Policy ⚠️

**What is Store Assignment Policy?**

Databricks SQL enforces **strict type checking** when writing data to tables:
* INSERT statements must match column types exactly
* CREATE TABLE AS SELECT (CTAS) infers types from the query
* Type mismatches cause errors, not silent conversions

**Why does this matter?**
* Prevents data corruption
* Ensures data quality
* Makes schema evolution explicit

**Common scenarios:**
* Inserting STRING into INT column
* Inserting larger precision into smaller precision
* NULL handling in NOT NULL columns

Let's see examples!

In [0]:
%sql
-- First, let's create a test table with specific types
-- We'll use this to demonstrate store assignment policy

CREATE OR REPLACE TABLE main.default.test_store_assignment (
  id INT,
  name STRING,
  amount DECIMAL(10, 2),
  created_date DATE
)
COMMENT 'Test table for store assignment policy demo'

In [0]:
%sql
-- This INSERT works - types match exactly

INSERT INTO main.default.test_store_assignment
VALUES 
  (1, 'Item A', 99.99, '2024-01-15'),
  (2, 'Item B', 149.50, '2024-01-16')

-- Check the data
SELECT * FROM main.default.test_store_assignment

In [0]:
%sql
-- This will FAIL - trying to insert STRING into INT column

INSERT INTO main.default.test_store_assignment
VALUES 
  ('three', 'Item C', 199.99, '2024-01-17')  -- 'three' is STRING, not INT

-- ERROR: Cannot safely cast 'three':STRING to id:INT

In [0]:
%sql
-- This will FAIL - decimal precision mismatch
-- Table expects DECIMAL(10,2) but we're providing more precision

INSERT INTO main.default.test_store_assignment
VALUES 
  (3, 'Item D', 199.999, '2024-01-17')  -- 3 decimal places, table expects 2

-- ERROR: Decimal precision/scale mismatch

In [0]:
%sql
-- Solution: Cast to the correct type

INSERT INTO main.default.test_store_assignment
VALUES 
  (CAST('3' AS INT), 'Item C', CAST(199.999 AS DECIMAL(10,2)), CAST('2024-01-17' AS DATE))

-- This works! The cast handles the conversion
SELECT * FROM main.default.test_store_assignment

In [0]:
%sql
-- CREATE TABLE AS SELECT (CTAS) infers types from the query
-- Be careful - the inferred types might not be what you expect!

CREATE OR REPLACE TABLE main.default.test_ctas_types AS
SELECT 
  1 AS int_col,                    -- Inferred as INT
  1.5 AS double_col,                -- Inferred as DOUBLE
  '2024-01-15' AS string_col,       -- Inferred as STRING (not DATE!)
  CAST('2024-01-15' AS DATE) AS date_col,  -- Explicitly DATE
  CURRENT_TIMESTAMP() AS timestamp_col     -- TIMESTAMP

-- Check the inferred types
DESCRIBE main.default.test_ctas_types

### ✅ Store Assignment Policy Best Practices

**1. Always specify types explicitly in CREATE TABLE:**
```sql
CREATE TABLE my_table (
  id INT NOT NULL,
  amount DECIMAL(10,2),
  name STRING
)
```

**2. Use CAST in INSERT statements:**
```sql
INSERT INTO my_table
VALUES (
  CAST(value AS INT),
  CAST(amount AS DECIMAL(10,2)),
  CAST(name AS STRING)
)
```

**3. Be explicit in CTAS:**
```sql
CREATE TABLE my_table AS
SELECT 
  CAST(id AS INT) AS id,
  CAST(amount AS DECIMAL(10,2)) AS amount,
  CAST(date_string AS DATE) AS date_col
FROM source_table
```

**4. Check types before inserting:**
```sql
DESCRIBE target_table  -- See expected types
DESCRIBE QUERY (SELECT ...)  -- See query result types
```

**⚠️ Key Takeaway:**
Databricks SQL will NOT silently convert types - you must be explicit!

## 5. SQL is Just an Abstraction for Spark! ⚡

**Key Insight:** Databricks SQL queries are compiled into Spark execution plans!

**What does this mean?**
* Your SQL query is translated into Spark operations
* Executed on the same Spark engine as PySpark/Scala
* Benefits from Spark optimizations (Catalyst, Photon)
* Can see the execution plan with `EXPLAIN`

**Why is this important?**
* Understanding the plan helps debug performance issues
* See what Spark is actually doing
* Identify optimization opportunities
* Compare different query approaches

Let's explore the execution plan!

In [0]:
%sql
-- EXPLAIN shows the execution plan for a query
-- Let's start with a simple query

EXPLAIN
SELECT 
  passenger_count,
  fare_amount
FROM samples.nyctaxi.trips
WHERE fare_amount > 50
LIMIT 10

### 📊 Understanding EXPLAIN Output

**The execution plan shows:**

1. **Physical Plan** - How Spark will actually execute the query
2. **Operations** - Each step in the execution (scan, filter, project, etc.)
3. **Data Flow** - Bottom-up execution (read from bottom to top)

**Common operations you'll see:**

* **Scan** - Reading data from a table
* **Filter** - WHERE clause conditions
* **Project** - SELECT column selection
* **HashAggregate** - GROUP BY aggregations
* **Sort** - ORDER BY sorting
* **Exchange** - Data shuffle between nodes
* **BroadcastExchange** - Broadcasting small tables

**Reading the plan:**
```
== Physical Plan ==
(3) GlobalLimit 10              ← LIMIT 10
(2) Filter (fare_amount > 50)   ← WHERE clause
(1) Scan table                  ← FROM clause (start here)
```

Read from bottom to top!

In [0]:
%sql
-- EXPLAIN for our aggregation query
-- This shows a more complex plan with shuffles

EXPLAIN
SELECT 
  HOUR(tpep_pickup_datetime) AS pickup_hour,
  COUNT(*) AS trip_count,
  ROUND(SUM(fare_amount), 2) AS total_revenue,
  ROUND(AVG(fare_amount), 2) AS avg_fare
FROM samples.nyctaxi.trips
WHERE 
  fare_amount > 0
  AND trip_distance > 0
GROUP BY HOUR(tpep_pickup_datetime)
ORDER BY pickup_hour

### 🔍 EXPLAIN Variations

Databricks SQL provides different EXPLAIN modes:

**1. EXPLAIN (default) - Physical Plan**
```sql
EXPLAIN
SELECT * FROM table
```
Shows the actual execution plan.

**2. EXPLAIN EXTENDED - Detailed Information**
```sql
EXPLAIN EXTENDED
SELECT * FROM table
```
Shows parsed logical plan, analyzed plan, optimized plan, and physical plan.

**3. EXPLAIN FORMATTED - Pretty Print**
```sql
EXPLAIN FORMATTED
SELECT * FROM table
```
Shows plan in a more readable tree format.

**4. EXPLAIN COST - Cost-Based Optimization**
```sql
EXPLAIN COST
SELECT * FROM table
```
Shows estimated costs for each operation.

Let's try EXPLAIN EXTENDED!

In [0]:
%sql
-- EXPLAIN EXTENDED shows all optimization stages

EXPLAIN EXTENDED
SELECT 
  HOUR(tpep_pickup_datetime) AS pickup_hour,
  COUNT(*) AS trip_count
FROM samples.nyctaxi.trips
WHERE fare_amount > 0
GROUP BY HOUR(tpep_pickup_datetime)
LIMIT 5

### 💡 Key Insights from EXPLAIN

**What to look for in execution plans:**

**1. Scan Operations**
* **ColumnarToRow** - Reading from columnar format (Parquet/Delta)
* **Partition filters** - Pushed down to storage
* **Data skipping** - Using statistics to skip files

**2. Shuffle Operations (Exchange)**
* **HashPartitioning** - Data redistributed by hash of key
* **RoundRobinPartitioning** - Even distribution
* **SinglePartition** - All data to one partition
* ⚠️ Shuffles are expensive - minimize them!

**3. Aggregations**
* **HashAggregate** - Hash-based aggregation (fast)
* **Partial** - Pre-aggregation before shuffle
* **Final** - Final aggregation after shuffle

**4. Optimizations**
* **Predicate pushdown** - Filters applied early
* **Column pruning** - Only read needed columns
* **Constant folding** - Evaluate constants at compile time
* **AQE** - Adaptive Query Execution optimizations

**🎯 Pro Tip:**
Compare EXPLAIN output for different query approaches to choose the most efficient one!

## 6. Create the Summary Table 💾

Now let's create our final table in Unity Catalog using **CREATE TABLE AS SELECT (CTAS)**.

**CTAS Benefits:**
* Creates table and inserts data in one statement
* Automatically infers schema from query
* Efficient - no separate INSERT needed
* Perfect for ETL pipelines

**Syntax:**
```sql
CREATE OR REPLACE TABLE catalog.schema.table_name AS
SELECT ...
```

In [0]:
%sql
-- Create the taxi hourly summary table
-- This combines all our transformations!

CREATE OR REPLACE TABLE main.default.taxi_hourly_summary AS
SELECT 
  HOUR(tpep_pickup_datetime) AS pickup_hour,
  COUNT(*) AS trip_count,
  ROUND(SUM(fare_amount), 2) AS total_revenue,
  ROUND(AVG(fare_amount), 2) AS avg_fare,
  ROUND(AVG(trip_distance), 2) AS avg_distance
FROM samples.nyctaxi.trips
WHERE 
  fare_amount > 0
  AND trip_distance > 0
  AND fare_amount < 500
GROUP BY HOUR(tpep_pickup_datetime)
ORDER BY pickup_hour

In [0]:
%sql
-- Verify the table was created successfully

SELECT * FROM main.default.taxi_hourly_summary
ORDER BY pickup_hour

In [0]:
%sql
-- Check the schema that was inferred from our query

DESCRIBE main.default.taxi_hourly_summary

In [0]:
%sql
-- See detailed table information

DESCRIBE EXTENDED main.default.taxi_hourly_summary

In [0]:
%sql
-- Let's see the execution plan for our complete query
-- This shows how Spark executes the entire transformation pipeline

EXPLAIN
SELECT 
  HOUR(tpep_pickup_datetime) AS pickup_hour,
  COUNT(*) AS trip_count,
  ROUND(SUM(fare_amount), 2) AS total_revenue,
  ROUND(AVG(fare_amount), 2) AS avg_fare,
  ROUND(AVG(trip_distance), 2) AS avg_distance
FROM samples.nyctaxi.trips
WHERE 
  fare_amount > 0
  AND trip_distance > 0
  AND fare_amount < 500
GROUP BY HOUR(tpep_pickup_datetime)
ORDER BY pickup_hour

### 🔍 Analyzing the Execution Plan

**In the plan above, you should see:**

1. **Scan** - Reading from `samples.nyctaxi.trips`
   * Columnar scan (efficient Parquet/Delta reading)
   * Partition pruning if applicable

2. **Filter** - WHERE clause conditions
   * `fare_amount > 0 AND trip_distance > 0 AND fare_amount < 500`
   * Applied early (predicate pushdown)

3. **Project** - Extracting HOUR and selecting columns
   * `HOUR(tpep_pickup_datetime)`

4. **HashAggregate (Partial)** - Pre-aggregation
   * Happens before shuffle
   * Reduces data movement

5. **Exchange** - Shuffle by pickup_hour
   * Data redistributed across executors
   * ⚠️ This is where skew could occur!

6. **HashAggregate (Final)** - Final aggregation
   * Completes the GROUP BY

7. **Sort** - ORDER BY pickup_hour
   * Final sorting step

**💡 Key Insight:**
This is the SAME execution plan you'd get with PySpark! SQL is just a different syntax for the same Spark operations.

## 🔄 SQL vs PySpark: Same Result, Different Syntax

Let's compare how we achieved the same result in both languages:

### **Complete Comparison**

| Operation | SQL | PySpark |
|-----------|-----|----------|
| **Read data** | `SELECT * FROM table` | `spark.table("table")` |
| **Select columns** | `SELECT col1, col2` | `.select("col1", "col2")` |
| **Filter rows** | `WHERE condition` | `.filter(condition)` |
| **Add column** | `CASE WHEN ... END AS col` | `.withColumn("col", when()...)` |
| **Extract hour** | `HOUR(timestamp)` | `hour("timestamp")` |
| **Aggregate** | `GROUP BY col` | `.groupBy("col").agg(...)` |
| **Count** | `COUNT(*)` | `count("*")` |
| **Sum** | `SUM(col)` | `sum("col")` |
| **Average** | `AVG(col)` | `avg("col")` |
| **Round** | `ROUND(col, 2)` | `round(col, 2)` |
| **Sort** | `ORDER BY col` | `.orderBy("col")` |
| **Create table** | `CREATE TABLE AS SELECT` | `.write.saveAsTable()` |
| **Explain plan** | `EXPLAIN query` | `df.explain()` |

### **Key Insight:**

Both SQL and PySpark compile to the **same Spark execution plan**! Choose based on:
* **SQL** - Familiar syntax, great for analytics, BI tool integration
* **PySpark** - Programmatic control, complex logic, Python ecosystem

## ✅ Databricks SQL Best Practices

### **1. Type Safety**

✅ Always use explicit CAST for type conversions  
✅ Check types with DESCRIBE before INSERT  
✅ Use DESCRIBE QUERY to verify result types  
❌ Don't rely on implicit type conversion  

### **2. Store Assignment Policy**

✅ Define explicit schemas in CREATE TABLE  
✅ Cast values in INSERT statements  
✅ Be explicit in CTAS with CAST  
❌ Don't assume type inference matches your intent  

### **3. Performance**

✅ Use EXPLAIN to understand query execution  
✅ Look for expensive shuffles (Exchange)  
✅ Minimize data movement  
✅ Use partition pruning when possible  
✅ Enable AQE for automatic optimization  

### **4. Unity Catalog**

✅ Use three-level namespace: `catalog.schema.table`  
✅ Leverage Unity Catalog for governance  
✅ Use SHOW commands to explore  
✅ Use DESCRIBE to understand schemas  

### **5. Query Development**

✅ Test with LIMIT during development  
✅ Use display() for interactive exploration  
✅ Add comments to complex queries  
✅ Use CTEs for readability  
✅ Compare EXPLAIN plans for optimization  

## ⚠️ Common Databricks SQL Gotchas

### **1. Strict Typing**

**Problem:**
```sql
SELECT '5' > 10  -- ERROR: Cannot compare STRING to INT
```

**Solution:**
```sql
SELECT CAST('5' AS INT) > 10  -- Works!
```

---

### **2. Store Assignment Policy**

**Problem:**
```sql
CREATE TABLE t (id INT);
INSERT INTO t VALUES ('123');  -- ERROR: STRING cannot be inserted into INT
```

**Solution:**
```sql
INSERT INTO t VALUES (CAST('123' AS INT));  -- Works!
```

---

### **3. Date/Timestamp Strings**

**Problem:**
```sql
CREATE TABLE t AS SELECT '2024-01-15' AS date_col;
-- date_col is STRING, not DATE!
```

**Solution:**
```sql
CREATE TABLE t AS SELECT CAST('2024-01-15' AS DATE) AS date_col;
-- Now date_col is DATE type
```

---

### **4. NULL Handling**

**Problem:**
```sql
CREATE TABLE t (id INT NOT NULL);
INSERT INTO t VALUES (NULL);  -- ERROR: NULL in NOT NULL column
```

**Solution:**
```sql
-- Either allow NULLs or provide default values
CREATE TABLE t (id INT);  -- Allows NULL
-- or
INSERT INTO t VALUES (COALESCE(NULL, 0));  -- Provide default
```

## 📚 Quick Reference Guide

### **Exploring Unity Catalog**
```sql
SHOW CATALOGS
SHOW SCHEMAS IN catalog_name
SHOW TABLES IN catalog.schema
DESCRIBE catalog.schema.table
DESCRIBE EXTENDED catalog.schema.table
```

### **Type Casting**
```sql
CAST(column AS INT)
CAST(column AS DECIMAL(10,2))
CAST(column AS DATE)
CAST(column AS TIMESTAMP)
column::INT  -- Shorthand
```

### **Aggregations**
```sql
COUNT(*), COUNT(DISTINCT col)
SUM(col), AVG(col)
MIN(col), MAX(col)
ROUND(col, decimals)
```

### **Date/Time Functions**
```sql
HOUR(timestamp), DAY(date), MONTH(date), YEAR(date)
DATE_TRUNC('hour', timestamp)
DATE_ADD(date, days)
DATEDIFF(date1, date2)
CURRENT_DATE(), CURRENT_TIMESTAMP()
```

### **Conditional Logic**
```sql
CASE 
  WHEN condition1 THEN value1
  WHEN condition2 THEN value2
  ELSE default_value
END
```

### **Table Operations**
```sql
CREATE OR REPLACE TABLE catalog.schema.table AS SELECT ...
INSERT INTO catalog.schema.table VALUES (...)
INSERT INTO catalog.schema.table SELECT ...
DROP TABLE catalog.schema.table
```

### **Debugging**
```sql
EXPLAIN query
EXPLAIN EXTENDED query
DESCRIBE QUERY (SELECT ...)
```

## 🎉 Congratulations!

You've completed the Databricks SQL Fundamentals demo!

### **What You Learned:**

✅ **SQL Transformations** - SELECT, WHERE, CASE, GROUP BY, ORDER BY  
✅ **Strict Typing** - Explicit casting required for type safety  
✅ **Store Assignment Policy** - Type enforcement in INSERT/CREATE  
✅ **EXPLAIN Plans** - SQL queries compile to Spark execution plans  
✅ **Unity Catalog** - Modern data governance and table management  
✅ **SQL = Spark** - Same engine, different syntax  

---

### **Key Takeaways:**

1. **Databricks SQL is Spark** - Your queries run on the same distributed engine
2. **Type safety matters** - Always cast explicitly to avoid errors
3. **EXPLAIN is your friend** - Use it to understand and optimize queries
4. **Unity Catalog is the standard** - Use three-level namespace
5. **SQL and PySpark are equivalent** - Choose based on your use case

---

### **Next Steps:**

* Practice writing complex SQL queries
* Explore advanced SQL functions (window functions, CTEs, etc.)
* Learn about Delta Lake features (MERGE, time travel, etc.)
* Study query optimization techniques
* Build SQL-based data pipelines

---

### **Resources:**

* [Databricks SQL Documentation](https://docs.databricks.com/sql/)
* [SQL Language Reference](https://docs.databricks.com/sql/language-manual/)
* [Unity Catalog Guide](https://docs.databricks.com/data-governance/unity-catalog/)
* [Query Optimization](https://docs.databricks.com/optimizations/)

---

**Happy querying!** 🚀